<a href="https://colab.research.google.com/github/tdubon/TF-GB-Forest/blob/main/TFClassificationForest_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow_decision_forests 

In [ ]:
!pip install gradio

In [ ]:
!pip install huggingface_hub

In [ ]:
from huggingface_hub import notebook_login
from huggingface_hub.keras_mixin import push_to_hub_keras

In [ ]:
import tensorflow_decision_forests as tfdf
import pandas as pd
import gradio as gr
import urllib

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


In [ ]:
notebook_login()

ERROR:root:HfApi.login: This method is deprecated in favor of `set_access_token`.


Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [ ]:
input_path = "https://archive.ics.uci.edu/ml/machine-learning-databases/census-income-mld/census-income"
input_column_header = "income_level"

In [ ]:
#Load data

BASE_PATH = input_path
CSV_HEADER = [ l.decode("utf-8").split(":")[0].replace(" ", "_")
  for l in urllib.request.urlopen(f"{BASE_PATH}.names")
  if not l.startswith(b"|")][2:]

CSV_HEADER.append(input_column_header)

train_data = pd.read_csv(f"{BASE_PATH}.data.gz", header=None, names=CSV_HEADER)
test_data = pd.read_csv(f"{BASE_PATH}.test.gz", header=None, names=CSV_HEADER)

In [ ]:
#subset data
train_data = train_data.loc[:, ["education", "sex", "capital_gains", "capital_losses", "income_level"]]
test_data = test_data.loc[:, ["education", "sex", "capital_gains", "capital_losses", "income_level"]]


In [ ]:
#train_data.shape

(199523, 5)

In [ ]:
#test_data["sex"].unique()

array([' Female', ' Male'], dtype=object)

In [ ]:
def encode_df(df):
    sex_mapping = {" Male": 0, " Female": 1}
    df = df.replace({"sex": sex_mapping})
    education_mapping = {" High school graduate": 1, " Some college but no degree": 2, 
                         " 10th grade": 3, " Children": 4, " Bachelors degree(BA AB BS)": 5, 
                         " Masters degree(MA MS MEng MEd MSW MBA)": 6, " Less than 1st grade": 7,
                         " Associates degree-academic program": 8, " 7th and 8th grade": 9,
                         " 12th grade no diploma": 10, " Associates degree-occup /vocational": 11,
                         " Prof school degree (MD DDS DVM LLB JD)": 12, " 5th or 6th grade": 13,
                         " 11th grade": 14, " Doctorate degree(PhD EdD)": 15, " 9th grade": 16,
                         " 1st 2nd 3rd or 4th grade": 17}
    df = df.replace({"education": education_mapping})
    income_mapping = {' - 50000.': 0, ' 50000+.': 1}
    df = df.replace({"income_level": income_mapping})
    return df

In [ ]:
train_data = encode_df(train_data)
test_data = encode_df(test_data)

In [ ]:
feature_a = tfdf.keras.FeatureUsage(name="education", semantic=tfdf.keras.FeatureSemantic.CATEGORICAL)
feature_b = tfdf.keras.FeatureUsage(name="sex", semantic=tfdf.keras.FeatureSemantic.CATEGORICAL)
feature_c = tfdf.keras.FeatureUsage(name="capital_gains", semantic=tfdf.keras.FeatureSemantic.CATEGORICAL)
feature_d = tfdf.keras.FeatureUsage(name="capital_losses", semantic=tfdf.keras.FeatureSemantic.CATEGORICAL)


In [ ]:
# Convert the dataset into a TensorFlow dataset.
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_data, label="income_level")
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_data, label="income_level")

/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core.py:2036: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)


In [ ]:
# Train a GB Trees model
model = tfdf.keras.GradientBoostedTreesModel(
      features = [feature_a, feature_b, feature_c, feature_d],
      exclude_non_specified_features = True,
      growing_strategy = "BEST_FIRST_GLOBAL",
      num_trees = 500,
      max_depth = 7,
      min_examples = 6,
      subsample = 0.5,
      sampling_method = "GOSS",
      validation_ratio = 0.1,
      task = tfdf.keras.Task.CLASSIFICATION,
      loss = "DEFAULT",
      verbose=0)
model.compile(metrics=[keras.metrics.BinaryAccuracy(name="accuracy")])


In [ ]:
#tf.autograph.experimental.do_not_convert(model.fit(train_ds))

In [ ]:
# Summary of the model structure.
#model.summary()

In [ ]:
model.fit(train_ds)

In [ ]:
# Evaluate the model.
#model.evaluate(test_ds, verbose=0)

# Export the model to a SavedModel.
#model.save("project/model")

In [ ]:
#prepare user input for the model
def process_inputs(edu_in, sex_in, cap_gains_in, cap_losses_in):
  df = pd.DataFrame.from_dict(
      {
          "education": [edu_in], 
          "sex": [sex_in],
          "capital_gains": [cap_gains_in],
          "capital_losses": [cap_losses_in]    
      }
  )
  df = encode_df(df)
  
  feature_a = tfdf.keras.FeatureUsage(name="education", semantic=tfdf.keras.FeatureSemantic.CATEGORICAL)
  feature_b = tfdf.keras.FeatureUsage(name="sex", semantic=tfdf.keras.FeatureSemantic.CATEGORICAL)
  feature_c = tfdf.keras.FeatureUsage(name="capital_gains", semantic=tfdf.keras.FeatureSemantic.CATEGORICAL)
  feature_d = tfdf.keras.FeatureUsage(name="capital_losses", semantic=tfdf.keras.FeatureSemantic.CATEGORICAL)
  
  df = tfdf.keras.pd_dataframe_to_tf_dataset(df)
  
  pred = model.predict(df)

  for i in pred:
    if i > .45:
      pred_bi = 1
      return {"> $50,000": pred_bi}
    else:
      pred_bi = 0
      return {"<= $50,000": pred_bi}


In [ ]:
iface = gr.Interface(
    process_inputs,
    [
     gr.inputs.Dropdown([" 1st 2nd 3rd or 4th grade", " High school graduate", 
                         " Bachelors degree(BA AB BS)", " Masters degree(MA MS MEng MEd MSW MBA)", 
                         " Prof school degree (MD DDS DVM LLB JD)",
                         " Doctorate degree(PhD EdD)"], type="index", label="education"), 
     gr.inputs.Radio([" Male", " Female"], label="Sex (M=0, F=1)", type="index"),
     gr.inputs.Slider(minimum = 0, maximum = 99999, label="capital gains"),
     gr.inputs.Slider(minimum = 0, maximum = 4608, label="capital losses")
    ],
    gr.outputs.Label(num_top_classes=2),
    live=True,
    analytics_enabled=False,
    examples=[
        [" Masters degree(MA MS MEng MEd MSW MBA)", 0, 7298, 0],
        [" Bachelors degree(BA AB BS)", 0, 6514, 0],
    ],
)

iface.launch(debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://21835.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.


(<fastapi.applications.FastAPI at 0x7f8a84dcde10>,
 'http://127.0.0.1:7860/',
 'https://21835.gradio.app')

In [ ]:
from huggingface_hub import KerasModelHubMixin
from huggingface_hub.keras_mixin import push_to_hub_keras
push_to_hub_keras(model, repo_url="https://huggingface.co/spaces/tdubon/TF-GB-Forest")